# Data Preparation 2

### Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

### Set up data directory

In [2]:
! pwd

/Users/vj/data/Projects/BaruchCollege/DataMiningForBusinessAnalytics


In [3]:
! ls -l | grep ^d

drwxr-xr-x  32 vj  staff     1024 Jan 18 07:52 data
drwxr-xr-x  24 vj  staff      768 Dec 29 09:23 data_old
drwxr-xr-x  19 vj  staff      608 Dec 12 10:17 images
drwxr-xr-x   4 vj  staff      128 Sep 23 15:33 models
drwxr-xr-x  27 vj  staff      864 Dec 26 16:51 old_notebooks
drwxr-xr-x   9 vj  staff      288 Oct  4 20:30 reference_images
drwxr-xr-x  17 vj  staff      544 Nov 10 13:56 reference_notebooks
drwxr-xr-x   7 vj  staff      224 Feb 22 21:09 slides


In [4]:
# mkdir data     # Only if it is not listed above

### To get original data file from Kaggle
* Go to https://www.kaggle.com/uciml/electric-power-consumption-data-set/metadata and click Download button
* Move downloaded file to data directory

### Note: A smaller version of the original file is in the data directory: household_power_consumption.txt

### Prepare smaller dataset

In [5]:
# Check how many data lines are in data file
! wc -l data/household_power_consumption.txt

  328697 data/household_power_consumption.txt


In [6]:
# Review first 5 data lines
! head -5 data/household_power_consumption.txt

In [7]:
# Review last 5 data lines
! tail -5 data/household_power_consumption.txt

In [8]:
# Load downloaded data file into data frame
df = pd.read_csv('data/household_power_consumption.txt', sep=';', low_memory=False)
df.shape

(328696, 9)

In [9]:
# From large dataset generate a smaller dataset (a random sample of 10,000 records)
df2 = df.sample(n = 10000, random_state = 42)
df2.shape

(10000, 9)

In [10]:
# Review first 10 records in data frame
df2.head(10)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
12913,25/12/2006,16:37:00,4.410,0.368,239.310,18.400,40.000,0.000,0.0
143703,26/3/2007,12:27:00,0.482,0.110,239.910,2.200,0.000,1.000,0.0
259284,14/6/2007,18:48:00,1.548,0.266,239.850,6.400,1.000,1.000,17.0
327569,1/8/2007,04:53:00,1.528,0.078,235.080,6.400,0.000,0.000,17.0
17476,28/12/2006,20:40:00,1.426,0.000,241.420,5.800,0.000,0.000,0.0
257409,13/6/2007,11:33:00,0.262,0.122,242.720,1.200,0.000,3.000,0.0
317574,25/7/2007,06:18:00,0.126,0.000,236.210,0.600,0.000,0.000,0.0
192853,29/4/2007,15:37:00,?,?,?,?,?,?,NaN
252340,9/6/2007,23:04:00,1.274,0.088,241.450,5.200,0.000,0.000,0.0
175812,17/4/2007,19:36:00,1.062,0.382,239.950,4.600,0.000,0.000,0.0


In [11]:
# Save smaller dataset to CSV file
df2.to_csv('data/hpc.csv')

In [12]:
# Load the newly created CSV file and review the smaller dataset
df3 = pd.read_csv('data/hpc.csv')
df3.head(10)

,Unnamed: 0,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,12913,25/12/2006,16:37:00,4.410,0.368,239.310,18.400,40.000,0.000,0.0
1,143703,26/3/2007,12:27:00,0.482,0.110,239.910,2.200,0.000,1.000,0.0
2,259284,14/6/2007,18:48:00,1.548,0.266,239.850,6.400,1.000,1.000,17.0
3,327569,1/8/2007,04:53:00,1.528,0.078,235.080,6.400,0.000,0.000,17.0
4,17476,28/12/2006,20:40:00,1.426,0.000,241.420,5.800,0.000,0.000,0.0
5,257409,13/6/2007,11:33:00,0.262,0.122,242.720,1.200,0.000,3.000,0.0
6,317574,25/7/2007,06:18:00,0.126,0.000,236.210,0.600,0.000,0.000,0.0
7,192853,29/4/2007,15:37:00,?,?,?,?,?,?,NaN
8,252340,9/6/2007,23:04:00,1.274,0.088,241.450,5.200,0.000,0.000,0.0
9,175812,17/4/2007,19:36:00,1.062,0.382,239.950,4.600,0.000,0.000,0.0


### Data imputation

In [13]:
# Check for null values
df3.isnull().sum()

Unnamed: 0                 0
Date                       0
Time                       0
Global_active_power        0
Global_reactive_power      0
Voltage                    0
Global_intensity           0
Sub_metering_1             0
Sub_metering_2             0
Sub_metering_3           135
dtype: int64

In [14]:
# Identify rows with missing values
df3[df3.isnull().any(axis=1)]

,Unnamed: 0,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
7,192853,29/4/2007,15:37:00,?,?,?,?,?,?,NaN
62,190784,28/4/2007,05:08:00,?,?,?,?,?,?,NaN
191,190782,28/4/2007,05:06:00,?,?,?,?,?,?,NaN
258,191317,28/4/2007,14:01:00,?,?,?,?,?,?,NaN
432,191013,28/4/2007,08:57:00,?,?,?,?,?,?,NaN
...,...,...,...,...,...,...,...,...,...,...
9686,190953,28/4/2007,07:57:00,?,?,?,?,?,?,NaN
9707,190787,28/4/2007,05:11:00,?,?,?,?,?,?,NaN
9774,191036,28/4/2007,09:20:00,?,?,?,?,?,?,NaN
9788,194187,30/4/2007,13:51:00,?,?,?,?,?,?,NaN


#### Note: Though several rows have '?' in several columns, those columns are not listed in 'isnull().sum()' output

In [15]:
# Replace '?' with NaN across all columns
df3['Global_active_power'].replace('?', np.nan, inplace = True)
df3['Global_reactive_power'].replace('?', np.nan, inplace = True)
df3['Voltage'].replace('?', np.nan, inplace = True)
df3['Global_intensity'].replace('?', np.nan, inplace = True)
df3['Sub_metering_1'].replace('?', np.nan, inplace = True)
df3['Sub_metering_2'].replace('?', np.nan, inplace = True)

In [16]:
# Check for null values
df3.isnull().sum()

Unnamed: 0                 0
Date                       0
Time                       0
Global_active_power      135
Global_reactive_power    135
Voltage                  135
Global_intensity         135
Sub_metering_1           135
Sub_metering_2           135
Sub_metering_3           135
dtype: int64

In [17]:
# Verify cells with '?' have been replaced with NaN
df3[df3.isnull().any(axis=1)]

,Unnamed: 0,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
7,192853,29/4/2007,15:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,190784,28/4/2007,05:08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,190782,28/4/2007,05:06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,191317,28/4/2007,14:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432,191013,28/4/2007,08:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9686,190953,28/4/2007,07:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9707,190787,28/4/2007,05:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9774,191036,28/4/2007,09:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9788,194187,30/4/2007,13:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Setup imputer to replace NaN cells with mean of column
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [19]:
# Apply imputer
X = df3[['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', \
         'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3' ]]
imp_mean.fit(X)

SimpleImputer()

In [20]:
# Create new dataframe and transform values
df4 = pd.DataFrame(imp_mean.transform(X), columns = ['Global_active_power', 'Global_reactive_power', \
                                                     'Voltage', 'Global_intensity', 'Sub_metering_1', \
                                                     'Sub_metering_2', 'Sub_metering_3'])

In [21]:
# Add back Date and Time columns to new dataframe
df4['Date'] = df3['Date']
df4['Time'] = df3['Time']
df4.head(20)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Date,Time
0,4.410000,0.368000,239.310000,18.400000,40.000000,0.000000,0.000000,25/12/2006,16:37:00
1,0.482000,0.110000,239.910000,2.200000,0.000000,1.000000,0.000000,26/3/2007,12:27:00
2,1.548000,0.266000,239.850000,6.400000,1.000000,1.000000,17.000000,14/6/2007,18:48:00
3,1.528000,0.078000,235.080000,6.400000,0.000000,0.000000,17.000000,1/8/2007,04:53:00
4,1.426000,0.000000,241.420000,5.800000,0.000000,0.000000,0.000000,28/12/2006,20:40:00
5,0.262000,0.122000,242.720000,1.200000,0.000000,3.000000,0.000000,13/6/2007,11:33:00
6,0.126000,0.000000,236.210000,0.600000,0.000000,0.000000,0.000000,25/7/2007,06:18:00
7,1.146193,0.124443,239.125355,4.901835,1.234161,1.736341,5.606285,29/4/2007,15:37:00
8,1.274000,0.088000,241.450000,5.200000,0.000000,0.000000,0.000000,9/6/2007,23:04:00
9,1.062000,0.382000,239.950000,4.600000,0.000000,0.000000,0.000000,17/4/2007,19:36:00


#### Note: Review row 7 above.  The NaNs have been replaced with its respective column mean.

In [22]:
# Check for null values
df4.isnull().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
Date                     0
Time                     0
dtype: int64

In [23]:
# Check for rows with any missing values
df4[df4.isnull().any(axis=1)]

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Date,Time


### Data standardization

In [24]:
df4['Global_intensity'].head(10)

0    18.400000
1     2.200000
2     6.400000
3     6.400000
4     5.800000
5     1.200000
6     0.600000
7     4.901835
8     5.200000
9     4.600000
Name: Global_intensity, dtype: float64

In [25]:
print(f"Global_intensity: min = {df4['Global_intensity'].min()}, max = {df4['Global_intensity'].max()}")

Global_intensity: min = 0.4, max = 37.0


In [26]:
# Standardize values
scaler = StandardScaler()

scaler.fit(np.array(df4[['Global_intensity']]))
df4['Global_intensity'] = scaler.transform(np.array(df4[['Global_intensity']]))

In [27]:
df4['Global_intensity'].head(10)

0    2.716400
1   -0.543723
2    0.301494
3    0.301494
4    0.180749
5   -0.744965
6   -0.865710
7    0.000000
8    0.060003
9   -0.060742
Name: Global_intensity, dtype: float64

In [28]:
print(f"Global_intensity: min = {df4['Global_intensity'].min()}, max = {df4['Global_intensity'].max()}")

Global_intensity: min = -0.9059588722502345, max = 6.459503527622711


### Data normalization

In [29]:
df4['Sub_metering_2'].head(10)

0    0.000000
1    1.000000
2    1.000000
3    0.000000
4    0.000000
5    3.000000
6    0.000000
7    1.736341
8    0.000000
9    0.000000
Name: Sub_metering_2, dtype: float64

In [30]:
print(f"Sub_metering_2: min = {df4['Sub_metering_2'].min()}, max = {df4['Sub_metering_2'].max()}")

Sub_metering_2: min = 0.0, max = 75.0


In [31]:
# Normalize values
normalizer = Normalizer()

normalizer.fit(np.array(df4[['Sub_metering_2']]))
df4['Sub_metering_2'] = normalizer.transform(np.array(df4[['Sub_metering_2']]))

In [32]:
df4['Sub_metering_2'].head(10)

0    0.0
1    1.0
2    1.0
3    0.0
4    0.0
5    1.0
6    0.0
7    1.0
8    0.0
9    0.0
Name: Sub_metering_2, dtype: float64

In [33]:
print(f"Sub_metering_2: min = {df4['Sub_metering_2'].min()}, max = {df4['Sub_metering_2'].max()}")

Sub_metering_2: min = 0.0, max = 1.0


#### In general, you should standardize data if you can safely assume it’s normally distributed.
#### Conversely, if you can safely assume that your data isn’t normally distributed, then normalization is a good method for scaling it.